# Ordering and Delivery

This notebook demonstrates ordering and download with the orders api. In this notebook, we check authentication by requesting an orders list, then we create an order for a list of visual `PSScene3Band` images. We request the order be delivered as a single zip file to an AWS s3 storage bucket.

Reference information can be found at [Ordering & Delivery](https://developers.planet.com/docs/orders/ordering-delivery/).

In [4]:
import json
import os
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth

import boto3
from botocore.exceptions import ClientError

import datetime as dt
import logging

In [6]:
# AWS account number.
ACCOUNT_NUMBER = '853558080719'

In [6]:
# today's date
today = dt.date.today()
date = today.strftime("%b-%d-%Y")
print("date =", date)

date = Mar-18-2021


In [23]:
scene_ids= ['20181031_095925_103b', '20200804_100556_1005', '20200501_100804_103c', '20200501_100803_103c', '20200501_100802_103c', '20200501_100801_103c', '20180718_184124_1040', '20180718_184123_1040', '20180607_082852_1012', '20190418_074358_0f3c', '20190503_083458_1001', '20161008_153105_0e3a', '20161008_153104_0e3a', '20160928_153233_0e16', '20181023_155054_1004', '20181024_160202_0f3b', '20181024_160201_0f3b', '20181022_155353_100a', '20181012_155937_0f2d', '20181023_155747_0f34', '20181023_155746_0f34', '20181022_155658_101e', '20181022_155659_101e', '20170227_203601_0c46', '20170227_203600_1_0c46', '20170227_203307_0c37', '20170323_152750_0e20', '20170322_153132_0e0f', '20170326_153240_0e26', '20170326_153239_0e26', '20170326_153233_0e26', '20170326_153234_0e26', '20170326_154021_0e2f', '20170826_154047_100c', '20170826_154046_100c', '20170825_170438_0f2a', '20170927_154611_0e14', '20170927_154610_0e14', '20171007_161315_0c65', '20171007_161314_1_0c65', '20171015_153720_100c', '20180107_153415_1032', '20180224_153959_0f1b', '20180224_153958_0f1b', '20180224_162902_0f2b', '20180313_154300_1008', '20180313_154259_1008', '20180313_154301_1008', '20180313_154258_1008', '20180919_160750_1020', '20180919_155029_1044', '20181124_155716_1049', '20181124_155715_1049', '20181124_155714_1049', '20181124_155713_1049', '20181215_075902_103e', '20181215_075901_103e', '20181215_075900_103e', '20181215_075859_103e', '20181215_075858_103e', '20181215_075857_103e']

In [38]:
len(scene_ids)

61

In [37]:
import json
import requests
from requests.auth import HTTPBasicAuth
ids = scene_ids

order_dict = {
  "name": f"marine_litter_order_{date}",
  "subscription_id": 0,
  "products": [
  {
      "item_ids": [],
      "item_type": "PSScene3Band",
      "product_bundle": "visual"
  }
  ],
  "delivery": {
      "single_archive": True,
      "archive_type": "zip",
      "archive_filename": f"marine_litter_order_{date}".zip",
      "amazon_s3": {
           "bucket": "marine-litter-observations",
           "aws_region": "us-east-1",
           "aws_access_key_id": os.environ['AWS_ACCESS_KEY'],
           "aws_secret_access_key": os.environ['AWS_SECRET_ACCESS_KEY'],
           "path_prefix": ""
        }
      },
  "notifications": {
  "email": True
  },
  "order_type": "full"
}


url = 'https://api.planet.com/compute/ops/orders/v2'
start_id = 0
responses = []
for end_id in range(0, len(ids), 50):
  selected_ids = ids[start_id:len(ids)]
  order_data = order_dict.copy()
  _id = len(ids) / 100
  order_data['name'] = order_data['name'].format(_id)
  order_data['products'][0]['item_ids'] = selected_ids
  order_data['delivery']['archive_filename'] = f"{order_data['name']}.zip"
  response = requests.post(url, auth=HTTPBasicAuth('<email>', '<password>'), json=order_data)
  responses.append(json.loads(response.content))
  start_id = end_id
  print(order_data['name'], response.status_code)

DEBUG: 2021-03-17 14:26:48,856: Starting new HTTPS connection (1): api.planet.com:443
DEBUG: 2021-03-17 14:26:51,199: https://api.planet.com:443 "POST /compute/ops/orders/v2 HTTP/1.1" 202 None
DEBUG: 2021-03-17 14:26:51,209: Starting new HTTPS connection (1): api.planet.com:443


marine_litter_order_03172020 202


DEBUG: 2021-03-17 14:26:52,978: https://api.planet.com:443 "POST /compute/ops/orders/v2 HTTP/1.1" 202 None


marine_litter_order_03172020 202
